# 🎨 Chapter A1: Texturing & UV Mapping

Welcome to **Advanced 3D Development**.

In the basics, we used **solid colors** for our shapes. Real games use **textures** (images wrapped around 3D objects).

This is how we go from colored cubes to photorealistic worlds.

## 1. What Are UV Coordinates?

**UV** is like a map that tells the GPU: "Which part of the image goes where on the 3D model?"

- **U** = Horizontal (0.0 = left, 1.0 = right)
- **V** = Vertical (0.0 = bottom, 1.0 = top)

Every vertex gets a position (X, Y, Z) **and** a UV coordinate (U, V).

## 2. Loading an Image Texture

We use **Pillow (PIL)** to load images into memory.

In [ ]:
from PIL import Image
import moderngl

# Load the image
img = Image.open('crate.png')
img = img.transpose(Image.FLIP_TOP_BOTTOM)  # OpenGL expects bottom-left origin

# Convert to bytes
img_data = img.convert('RGB').tobytes()

# Create OpenGL texture
# ctx is your ModernGL context
texture = ctx.texture(img.size, 3, img_data)
texture.filter = (moderngl.LINEAR, moderngl.LINEAR)  # Smooth filtering

## 3. Vertex Data with UV Coordinates

Now each vertex has **5 floats**: X, Y, Z, U, V

In [ ]:
import numpy as np

# Textured quad (two triangles)
# Format: X, Y, Z, U, V
quad_vertices = np.array([
    # Triangle 1
    -1.0, -1.0, 0.0,   0.0, 0.0,  # Bottom-left
     1.0, -1.0, 0.0,   1.0, 0.0,  # Bottom-right
     1.0,  1.0, 0.0,   1.0, 1.0,  # Top-right
    
    # Triangle 2
    -1.0, -1.0, 0.0,   0.0, 0.0,  # Bottom-left
     1.0,  1.0, 0.0,   1.0, 1.0,  # Top-right
    -1.0,  1.0, 0.0,   0.0, 1.0,  # Top-left
], dtype='f4')

vbo = ctx.buffer(quad_vertices.tobytes())

## 4. The Textured Shader

The vertex shader passes UV to the fragment shader, which samples the texture.

In [ ]:
vert_shader = '''
    #version 330
    in vec3 in_vert;
    in vec2 in_uv;
    
    uniform mat4 mvp;
    
    out vec2 v_uv;
    
    void main() {
        gl_Position = mvp * vec4(in_vert, 1.0);
        v_uv = in_uv;
    }
'''

frag_shader = '''
    #version 330
    in vec2 v_uv;
    
    uniform sampler2D u_texture;
    
    out vec4 f_color;
    
    void main() {
        f_color = texture(u_texture, v_uv);
    }
'''

prog = ctx.program(vertex_shader=vert_shader, fragment_shader=frag_shader)

# VAO: Tell GPU how to interpret the data
# '3f 2f' means: 3 floats for position, 2 floats for UV
vao = ctx.vertex_array(prog, [(vbo, '3f 2f', 'in_vert', 'in_uv')])

## 5. Rendering with Texture

Bind the texture before rendering.

In [ ]:
# In your render loop:

# Bind texture to slot 0
texture.use(0)
prog['u_texture'].value = 0

# Set MVP matrix
prog['mvp'].write(mvp.astype('f4').tobytes())

# Render
ctx.clear(0.1, 0.1, 0.1)
vao.render()
pygame.display.flip()

## 6. Texture Filtering & Mipmaps

When textures are viewed from far away, they can look pixelated or "shimmer". **Mipmaps** solve this.

In [ ]:
# Generate mipmaps (smaller versions of the texture)
texture.build_mipmaps()

# Use trilinear filtering for best quality
texture.filter = (moderngl.LINEAR_MIPMAP_LINEAR, moderngl.LINEAR)

## 7. Texture Atlases (Multiple Textures in One Image)

Loading 100 textures = 100 GPU state changes = SLOW.

Solution: Pack all textures into one big image and use different UV ranges.

In [ ]:
# Example: 4 textures in a 2x2 grid
# Texture 1 (top-left): UV = (0.0-0.5, 0.5-1.0)
# Texture 2 (top-right): UV = (0.5-1.0, 0.5-1.0)
# Texture 3 (bottom-left): UV = (0.0-0.5, 0.0-0.5)
# Texture 4 (bottom-right): UV = (0.5-1.0, 0.0-0.5)

# Adjust UV coordinates accordingly
grass_uv = [0.0, 0.0, 0.5, 0.5]  # Bottom-left quadrant
stone_uv = [0.5, 0.0, 1.0, 0.5]  # Bottom-right quadrant

## 🛠️ Challenge: The Textured Room

Build a 3D room with different textures on each wall:

1. Load 4 different textures (or use a texture atlas)
2. Create a cube where each face has a different texture
3. Position the camera **inside** the cube
4. Use WASD to walk around and look at the walls

**Bonus:** Add a floor texture that tiles (repeats) by using UV coordinates > 1.0

If you can walk around a textured room, you've mastered UV mapping. 🏠